In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
from pathlib import Path
import openpyxl
import itertools
import torch

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

from threadpoolctl import threadpool_limits, threadpool_info


Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Python 3.12 and above currently is not supported by Spark and Ray. Please note that some functionality will not work and currently is not supported.


### Read Datasets

In [2]:
borderline_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.parquet") 
borderline2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.parquet")
smote_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.parquet")
smote2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.parquet")
gan_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_data.parquet")
gan2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_data.parquet")

original_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet")

In [ ]:
print(datacompy.Compare(original_df, borderline_df, join_columns='Label').report())

In [ ]:
borderline_df.drop_duplicates(inplace=True)

In [ ]:
original_df["Label"].value_counts().sum()

In [ ]:
print(smote_df["Label"].value_counts())

In [ ]:
print(smote_df["Label"].value_counts())
print(borderline_df["Label"].value_counts())

In [3]:
mix_df = pd.concat([borderline_df, borderline2_df, smote_df, smote2_df, gan_df, gan2_df], ignore_index=True)
smote_df = pd.concat([smote_df, smote2_df], ignore_index=True)
gan_df = pd.concat([gan_df, gan2_df], ignore_index=True)
borderline_df = pd.concat([borderline_df, borderline2_df], ignore_index=True)

In [4]:
print(original_df["Label"].value_counts())
count1=original_df["Label"].value_counts().sum()
count2=abs((original_df['Label']==0).sum() - (original_df['Label']==1).sum())
print(f"Before undersampling: {count2}")
print(f"After number of samples: {count1+count2}")

Label
0    250832
1     62708
Name: count, dtype: int64
Before undersampling: 188124
After number of samples: 501664


In [7]:
mix_df["Label"].value_counts()

Label
1    1693116
Name: count, dtype: int64

In [5]:
#Split data
X_mix, y_mix = mix_df.drop(columns=["Label", "source"]), mix_df["Label"]
X_smote, y_smote = smote_df.drop(columns=["Label", "source"]), smote_df["Label"]
X_GAN, y_GAN = gan_df.drop(columns=["Label", "source"]), gan_df["Label"]
X_borderline, y_borderline = borderline_df.drop(columns=["Label", "source"]), borderline_df["Label"]

#Dictionary
data = {}
data["mix"] = (X_mix, y_mix)
data["smote"] = (X_smote, y_smote)
data["GAN"] = (X_GAN, y_GAN)
data["borderline"] = (X_borderline, y_borderline)

compare = {}
compare["mix"] = mix_df
compare["smote"] = smote_df
compare["GAN"] = gan_df
compare["borderline"] = borderline_df

### Undersampling

In [6]:
base = Path("D:/ml/undersampling_data/data/cicunsw/reduced")
#base2 = Path("D:/ml/undersampling_data/models/kmeans&centroids2")

df_ = {}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class
    X_majority = X_train[y_train == 1]
    
    KM = KMeans(n_clusters=(int)(count2))
    with threadpool_limits(limits=16):
    # Apply KMeans clustering
        kmeans = KM.fit(X_majority)
    
    # Create a DataFrame for centroids
    X_majority_reduced = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    y_majority_reduced = pd.Series([1] * (int)(count2), name="Label") 
    
    # Combine reduced majority class with original minority class
    df_majority = pd.concat([X_majority_reduced, y_majority_reduced], axis=1).reset_index(drop=True)
    
    # Add source column if not present
    df_majority["source"] = None
    missing_source = df_majority[df_majority["source"].isna()]
    if not missing_source.empty:
        df_majority.loc[df_majority["source"].isna(), "source"] = "centroid" 
    
    print(df_majority)
    df_majority = df_majority.reindex(columns=original_df.columns, fill_value=0.0)
    
    # Combine with original minority class
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)  
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_centroids.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Nie można odnaleźć określonego pliku
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close

KeyboardInterrupt: 

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
#base2 = Path("D:/ml/undersampling_data/models/kmeans&nn")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_NN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply KMeans clustering
    KM = KMeans(n_clusters=(int)(count2))
    kmeans = KM.fit(X_majority)
      
    # Centroids + Add rows to dictionary
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    # Original minority class
    for i in range(len(centroids_)):
        if (len(cluster_data_[name][i])>1):     # If more than one sample in the cluster
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = euclidean(centroid, row)         # Colculate Euclidean distance
                
            min_key = min(dist_, key=dist_.get)
            results_KM_NN_[name][i] = cluster_data_[name][i].iloc[[min_key]]    # Select the nearest neighbor to the centroid
            
        else:           # If only one sample in the cluster
            results_KM_NN_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_NN_[name].values(), ignore_index=True)    
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1] * (int)(count2), name="Label") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_nn.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")